In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from shutil import copy
import cv2
import os
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import applications

In [ ]:
df = pd.read_csv('../input/dog-breed-identification/labels.csv')
df.head()

In [ ]:
df['id'] = [i+'.jpg' for i in df['id']] 
df.head()

In [ ]:
def preprocess(p):
  i = cv2.imread(p)
  i = cv2.cvtColor(i, cv2.COLOR_BGR2RGB)
  i = cv2.resize(i, (400,400))
  return i

In [ ]:
! mkdir './train data'

In [ ]:
l = os.listdir('../input/dog-breed-identification/train')

for i in range(len(l)):
  path = '../input/dog-breed-identification/train/' + l[i]
  image = preprocess(path)
  cv2.imwrite('./train data/'+l[i], image)

In [ ]:
idg = ImageDataGenerator(rescale = 1/255.0, rotation_range=90,
                        width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.25,
                                   horizontal_flip=True,
                                   fill_mode='nearest', validation_split=0.1)

In [ ]:
bs = 64

In [ ]:
train_gen = idg.flow_from_dataframe(dataframe=df, directory='./train data', x_col='id', y_col='breed',
                                    target_size=(400,400),
                                    batch_size=bs,subset='training')

In [ ]:
val_gen = idg.flow_from_dataframe(dataframe=df, directory='./train data', x_col='id', y_col='breed',
                                   target_size=(400,400), batch_size=bs,subset='validation')

In [ ]:
transfer_model = applications.InceptionV3(include_top = False,
                                   weights ='imagenet',
                                  input_shape = (400,400,3))

In [ ]:
for layer in transfer_model.layers:
  layer.trainable = False

In [ ]:
model = tf.keras.models.Sequential()
model.add(transfer_model)
model.add(tf.keras.layers.MaxPool2D())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1024,activation='relu'))
model.add(tf.keras.layers.Dense(512,activation='relu'))
model.add(tf.keras.layers.Dense(120,activation = 'softmax'))

model.summary()

In [ ]:
callback = [tf.keras.callbacks.EarlyStopping(monitor='val_acc',patience=3)]

In [ ]:
model.compile(optimizer='SGD', loss=tf.keras.losses.categorical_crossentropy, metrics='accuracy')

In [ ]:
hist = model.fit(train_gen, batch_size=bs, epochs=25, validation_data=val_gen, callbacks=[callback])

In [ ]:
label = list(train_gen.class_indices)

In [ ]:
img = '../input/dog-breed-identification/test/'+os.listdir('../input/dog-breed-identification/test')[1]

In [ ]:
x = np.expand_dims(preprocess(img), axis=0)

In [ ]:
model.predict(x)

In [ ]:
pred = np.argmax(model.predict(x))
print("Breed:",label[pred])

In [ ]:
submission = pd.DataFrame(columns = label)

In [ ]:
submission.insert(0, 'id', os.listdir('../input/dog-breed-identification/test'))

In [ ]:
submission.head()

In [ ]:
submission.iloc[0, 1:]

In [ ]:
test_ids = os.listdir('../input/dog-breed-identification/test')
n = 0

for i in test_ids:
    image_path = '../input/dog-breed-identification/test/'+i
    image = np.expand_dims(preprocess(img), axis=0)
    submission.iloc[n, 1:] = model.predict(x)
    n+=1

In [ ]:
submission.head()